In [1]:
from transformers.utils import logging
# logging.set_verbosity(40)
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [2]:
import re

In [3]:
# для дообучения взяла абстрактный русский реп

text = """
Сегодня ты бородатый модник, похожий на Девендру Банхарта
А завтра разгоняешь геев с православными соратниками
Жизнь становится веселой, но ты все равно выглядишь жалко
Сковородки дома плавают в жирной немытой раковине
И некуда податься, чтобы оправдать существование
Этих муниципальных депутатов, этих рабочих в касках
Они спиливают аварийные деревья у твоего подъезда
Попробуй возненавидеть по приказу приходящую смерть
На полотенце следы от соуса, на полу макароны и пыль
Ты закрываешь дверь и спешишь свое фото выложить
С решительным лицом в Инстаграм - это инструмент политики
Ты можешь быть нашистом, потом хозяина возненавидеть
Тебе не будут мстить, даже возьмут на работу в СМИ
На остывающих валунах ада шипит пена огнезащитная
Это Россия, где перед тобой, сынок, все двери открыты
Выбирай судьбу героя одиночки или компанейской гниды
Ходи на оба митинга, у тебя сердце искателя приключений
Принципы придумали слабаки, тусовку придумали гении
Пей до дна за победу, за нашу и вашу свободу, смейся
Или проклинай неуклюжих ментов в армяках серых
Свобода - это дерзновение, свобода приходит нагая
Сегодня накати водочки, джинсы свои завтра постираешь
Поглубже вдохни этого воздуха, его не запретили пока
Отпизженный лимоновцами Удальцов смотрит на тебя свысока
Испытующий взгляд превращает
Лучи рассвета в полицейские дубинки
Время года сменяет другое
Распускаются цветы как не из этого мира
Еще верю во что-то, провожая юность
Встречая утро в июне
Как Брюс Уиллис в том фильме, еще не знаю
Но уже догадываюсь, что умер
Испытующий взгляд превращает Лучи рассвета в полицейские дубинки
Машины гудят где-то, стёкла ловят блики
Кулаки и флаги, как цветы, тянутся к небу
Еще верю во что-то, провожая юность
Встречая утро в июне
Меня тащат по асфальту куда-то
И я отдаю себя им в руки

Трусы продлевают себе жизнь плохо сложенными стихами
Топчутся по истине, собственные правила нарушая
Одна рука строчит памфлеты, другая - о прелести мещанства
Неудовлетворенность на родственниках вымещают
Однажды свежим утром я пришел на одно собрание
Предварительно начитался блогов, сформировал мнение
Спиздил у Шаргунова бейджик "писатель", собрал все обаяние
И пробрался к микрофону - "Вот что сказать я имею"
Как девственник перед шлюхой, перед аудиторией робкий
Но за мной стоит правда, и слова ко мне приходят
Вчера я клеил обои и ковырялся в своем огороде
А сегодня я правозащитник, политик, талант из народа
Паразитам пришло время спать на тонких матрасах
Должны жить они не во дворцах и ездить не в лимузинах
Я хочу их познакомить с очередями в магазинах
И для деловых поездок выдать велик "Урал" с мигалкой
Я говорил минут десять и не понял многое сам
Но услышал аплодисменты и восторженные голоса
Меня подхватили ребята, какие-то грязные бородачи
Давай к нам в палатку, ты наш новый мессия, ты чист
Мне протягивают стакан, и мы пьем допьяна
Обнимаемся и кричим: "Руки прочь, это наша страна!
Мы не уйдем отсюда!" Дубинки, пиздюли, пистолеты
Я тихонько блюю и уползаю навстречу рассвету

Испытующий взгляд превращает
Лучи рассвета в полицейские дубинки
Машины гудят где-то, стёкла ловят блики
Кулаки и флаги, как цветы, тянутся к небу
Еще верю во что-то провожая юность
Встречая утро в июне
Меня тащат по асфальту куда-то
И я отдаю себя им в руки
Испытующий взгляд превращает
Лучи рассвета в полицейские дубинки
Время года сменяет другое
Распускаются цветы как не из этого мира
Еще верю во что-то, провожая юность
Встречая утро в июне
Как Брюс Уиллис в том фильме, еще не знаю
Но уже догадываюсь, что я умер
"""

In [4]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64,
                            overwrite_cache=True)

# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
%pip install transformers[torch]

In [7]:
%pip install accelerate -U

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(

    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=1.0656546020507813, metrics={'train_runtime': 53.077, 'train_samples_per_second': 26.377, 'train_steps_per_second': 1.884, 'total_flos': 26129203200000.0, 'train_loss': 1.0656546020507813, 'epoch': 57.14})

In [13]:
text = "Невозможно больше"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Невозможно больше жить на свете, чем я"
Испытующий взгляд превращает
Лучи рассвета в полицейские дубинки
Время года сменяет другое
А я все продолжаю любить свою работу
И думать о том, как круто было бы встретить Новый год
Встретить его у подъезда своего дома
Улыбнуться и сказать спасибо за то, что ты есть
За то время, которое ты провел со мной
Тебя не сломали, не вышвырнули вон
Ты просто живешь сегодняшним днем
Судьба преподнесла тебе еще один урок
Неудачный день прошел, а ты все еще веришь в чудеса
Жизнь удалась, и ты снова готов к подвигам
Но судьба распорядилась по-другому
Поверь мне, завтра все может сложиться иначе
Заблудиться в своем лесу или затеряться среди звезд
На твоем месте я поступил бы точно так же
Отпизженный до полусмерти неудачник
Или пошел на попятный и признался сам себе
Что устал от бремени власти и вынужден довольствоваться ролью наемного работника
Вместо того чтобы бороться за свои права, ты строишь из себя героя
Выбираешь между карьерой и потребностью самовыраже

In [14]:
text = "Дедушка меня просил поговорить с одноклассниками"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дедушка меня просил поговорить с одноклассниками, но я отказался», - заявил он в интервью телеканалу «Россия-24».

Президент России Владимир Путин подписал закон о внесении изменений в Кодекс об административных правонарушениях (КоАП) и отдельные законодательные акты РФ в связи с принятием Федерального закона "О противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма". Документ опубликован на официальном интернет-портале правовой информации. Законопроектом вносятся изменения в Уголовный кодекс РФ, предусматривающие ответственность за распространение сведений, порочащих честь, достоинство или деловую репутацию других лиц, либо ущемляющих права и законные интересы граждан, юридических лиц и индивидуальных предпринимателей", - говорится в пояснительной записке к документу. В частности, предлагается ужесточить наказание для тех, кто распространяет информацию, способствующую разжиганию религиозной, расовой, национальной или религиозной враж

In [15]:
text = "Это я муравей, ползающий по изображению в макросъёмке твоего тела"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Это я муравей, ползающий по изображению в макросъёмке твоего тела
Ты читаешь мои мысли и смотришь на меня свысока
Я хочу быть с тобой, но ты строишь из себя слона
Тебе не нравится моя одежда, твои волосы дышат серой
Меня тащат по асфальту куда-то
И слова застревают у тебя в горле
Мне плевать, что подумают обо мне другие
Моя жизнь - игра, где нужно переигрывать
Кулаки и флаги, как цветы, тянутся к небу
Машины гудят где- то, стёкла ловят блики
Лучи рассвета озаряют улицы
Слезы катятся по щекам, когда я их изливаю
Пальцы скользят по клавиатуре, рисуя мелом замысловатые узоры
На мониторе высветлитель превращает текст в размытое пятно
Руки стирают краску с монитора, оставляя грязные разводы
Время года сменяет другое, а я все равно продолжаю жить
Трусы и лицемеры, вытирающие пот со лба
Выбирайте между добром и злом, выбирайте свободу самовыражения
Жизнь коротка, чтобы тратить ее на жалкие делишки
Встречая утро нового дня, первым делом вспоминай о вчерашнем дне
Отправляясь на очередную трени